In [45]:
import numpy as np
from IPython.display import Audio
import cv2
from matplotlib import pyplot as plt
import time

<center><h1>Compression d'image</h1></center>

# Les fonctions de la méthode mathématique (Utilisable en tout lieux)

In [46]:
# Norme (produit scalaire d'un vecteur avec lui même)
def norm(u):
    return np.sqrt(scalarProduct(u,u))

# Coordonées de la projection d'un vecteur v sur une famille orthonormée u
# Retourne une liste de scalaire de même longueur que la taille de u
def coordonnees(u,v):
    c = []
    for i in range(len(u)):
        c.append(scalarProduct(u[i],v))
    return c

# Projection orthogonale de v sur la famille u, qui utilise les coordonnées calculées précédemment
# Et qui renvoie c et la projection orthogonale
def projectionOrt(u,v):
    c = coordonnees(u,v)
    p = np.zeros_like(v)
    for i in range(len(u)):
        p += c[i]*u[i]
    return [c,p]


# Les fonctions propre au contexte de la compression d'image (espace vectoriel, famille de référence et produit scalaire)

In [52]:
# Produit scalaire 
def scalarProduct(u,v):
    sum_w = 0
    for x in range(len(u)):
        for y in range(len(u[0])):
            for z in range(len(u[0][0])):
                sum_w += (u[x][y][z]*v[x][y][z])
    return sum_w
    
# Notre famille de référence
def createOrthonormalBasis():
    v= []
    u=(np.load("base8.npz")["base"].astype(np.float64)/127.5-1)
    
    for i in range(0,50):
        sum_proj = 0
        for j in range(0,i):
            sum_proj += (scalarProduct(u[i],v[j]) / norm(v[j])**2)*v[j]
        
        v.append((u[i]- sum_proj)/(norm(u[i]- sum_proj)))
    
    return v

# Résolution des images
W=200 

def lireImage(file):
    bgr8b=cv2.imread(file) # Lecture du fichier
    bgr200=cv2.resize(bgr8b, (W,W)) # Redimensionnement
    rgb200=cv2.cvtColor(bgr200, cv2.COLOR_BGR2RGB) 
    # OpenCV range les octets en B, V, R. L'affichage les veut en R, V, B
    rgb200fl=rgb200.astype(np.float64)/127.5-1 # Valeurs entre -1 et 1 plutot que entre 0 et 255
    # Notez que ce n'est pas strictement nécessaire. De toutes façons, notre espace vectoriel est celui des images    #
    # dont les valeurs valent entre -∞ et +∞ (forcément, on peut de toutes façons tout multiplier par n'importe quel
    # scalaire)
    # L'avantage de se placer entre -1 et 1, c'est qu'au moins les images négatives continuent à avoir du sens pour 
    # nous
    # Le cvtColor est là parce que par défaut les images sont lues en bleu-vert-rouge, alors que les fonctions 
    # d'affichage supposent que c'est du rouge-vert-bleu
    # Le .astype(np.float64) est pour manipuler des réels et non des entiers comme valeurs    
    return rgb200fl

# L'affichage pourrait être plus simple. Mais comme vous avez souvent envie de faire des animations,
# en passant par ces fonctions, vous le pouvez.
# Vous devez appeler initAffichage une fois au début de la cellule qui va faire des affichages
# Puis affichage(image200x200, "un texte") à chaque image à afficher
# Utilisez "affichage" avec parcimonie, car cela prend un bon 1/4 de seconde 
def initAffichage():
    global imageAffich, figAffich
    figAffich=plt.figure(figsize=(8,8))
    imageAffich=plt.imshow(np.zeros((W,W,3), dtype=np.float64))
    
def affichage(img, titre=''):
    global imageAffich, figAffich
    imageAffich.set_data((img*0.5+0.5).clip(0,1))
    figAffich.canvas.draw()
    plt.title(titre)
    time.sleep(0.1)

# Démonstration
On test ici la fiabilité des différentes fonctions crée précédemment.

In [ ]:
# On crée notre espace vectoriel
u = createOrthonormalBasis()

# On va ici récupérer une image
#picture = lireImage("Pessi-lionel-messi-595x375.jpg")

# On récupère les coordonnées de notre son dans la base ainsi que sa projection orthonormée
picture_coeff, picture_product = projectionOrt(u, u[4])

#On affiche l'image d'origine
initAffichage()
affichage(picture,"Image d'origine")

#On affiche la projection orthogonale pour vérifier le fonctionnement correcte de notre code
initAffichage()
affichage(picture_product,"Projeté de l'image dans notre base")

v=(np.load("base8.npz")["base"].astype(np.float64)/127.5-1)

print("--- Norme ---")
print(norm(u[0]))
print(norm(u[1]))

print("")
print("--- Orthogonalité ---")
print( scalarProduct(u[0]/norm(u[0]),v[0]/norm(v[0])))
print(scalarProduct(u[1],u[0]))
print("")
print(scalarProduct(u[2],u[0]))
print(scalarProduct(u[2],u[1]))
print("")
print(scalarProduct(u[3],u[0]))
print(scalarProduct(u[3],u[1]))
print(scalarProduct(u[3],u[2]))
print("")
print(scalarProduct(u[4],u[0]))
print(scalarProduct(u[4],u[1]))
print(scalarProduct(u[4],u[2]))
print(scalarProduct(u[4],u[3]))

